In [8]:
from model.model_fn import ModelWrapper
import tensorflow as tf
import numpy as np

# Ideas

* bigrams as concepts

In [9]:
endpoints = dict(
    input_='model/dim_reduction', 
    bottleneck='model/bottleneck/BiasAdd',
    probs='model/Softmax',
    output='model/output_logits/BiasAdd'
)

mw = ModelWrapper('experiments/test/', endpoints)

graph = tf.get_default_graph()

sess = tf.Session(graph=graph)
sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from experiments/test/model.ckpt-955


In [6]:
text = 'обама прибыл в оон'
text2 = 'обама пошел в магазин'
labels = np.array([0])  # some pseudo-label

In [7]:
preds = mw.calculate_output(sess, text)

In [8]:
preds.shape  # 48 classes

(1, 48)

In [9]:
grads = mw.calculate_grad(sess, text2, labels)

In [10]:
grads.shape  # 256 is the size of the hidden layer (bottleneck)

(1, 256)

# Concepts

In [11]:
import pandas as pd
from model.utils import clean
import numpy as np

In [12]:
# data = pd.read_csv('data/concept_search.csv', nrows=None)

# concepts = data[data['text'].str.contains('Путин')].sample(500)['text'].apply(clean).apply(lambda x: x.replace('\xa0', ' '))
# random_data = data.sample(500)['text'].apply(clean).apply(lambda x: x.replace('\xa0', ' '))
# np.save('data/concepts.npy', concepts.values)
# np.save('data/random_data.npy', random_data.values)

In [13]:
concepts = np.load('data/concepts.npy')
random_data = np.load('data/random_data.npy')

In [14]:
%%time

X_conc = mw.calculate_bottleneck(sess, concepts)

X_rand = mw.calculate_bottleneck(sess, random_data)

CPU times: user 984 ms, sys: 199 ms, total: 1.18 s
Wall time: 413 ms


In [15]:
X = np.append(X_conc, X_rand, axis=0)
y = np.array([1] * len(X_conc) + [0] * len(X_rand))

In [16]:
np.save('data/X.npy', X)
np.save('data/y.npy', y)

# CAVs

In [1]:
import numpy as np
from cav import CAV

In [2]:
X = np.load('data/X.npy')
y = np.load('data/y.npy')

In [3]:
cav = CAV()

In [4]:
v = cav.fit(X, y)

In [5]:
np.save('data/cav.npy', v)

In [6]:
v.shape

(256,)

# TCAV

In [7]:
import pandas as pd
from model.model_fn import ModelWrapper
import tensorflow as tf
import numpy as np

In [8]:
endpoints = dict(
    input_='model/dim_reduction', 
    bottleneck='model/bottleneck/BiasAdd',
    probs='model/Softmax',
    output='model/output_logits/BiasAdd'
)

mw = ModelWrapper('experiments/test/', endpoints)

graph = tf.get_default_graph()

sess = tf.Session(graph=graph)
sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from experiments/test/model.ckpt-955


In [3]:
# train = pd.read_csv('data/48labs/train.csv')

# labs = train['labels'].values
# texts = train['text'].tolist()

In [5]:
%%time

grads = mw.calculate_grad(sess, texts, labs)

CPU times: user 4min 23s, sys: 44.7 s, total: 5min 8s
Wall time: 1min 38s


In [7]:
#  np.save('data/train_grads.npy', grads)

In [9]:
grads = np.load('data/train_grads.npy')
v = np.load('data/cav.npy')

In [14]:
dot_prod = grads.dot(v)

In [17]:
(dot_prod > 0).sum()

96508